# Rewriting/Reorganizing the codebase for EnKF

## Farsite

### Config File

In [30]:
import datetime
import os

from shapely.geometry import Polygon

from dsfunctions import get_observation


class Config_File:
    def __init__(self, 
                 FARSITE_START_TIME: datetime.datetime, 
                 FARSITE_END_TIME: datetime.datetime, 
                 windspeed: int, winddirection: int):
        self.__set_default()
        
        # Set the parameters
        self.FARSITE_TIMESTEP = int((FARSITE_END_TIME - FARSITE_START_TIME).total_seconds()/60)
        self.FARSITE_START_TIME = datetime.datetime(2019, 9, 9, 19, 0)
        self.FARSITE_END_TIME = self.FARSITE_START_TIME + (FARSITE_END_TIME - FARSITE_START_TIME)
        self.windspeed = windspeed
        self.winddirection = winddirection

    def __set_default(self):
        self.version = 1.0
        self.FARSITE_DISTANCE_RES = 30
        self.FARSITE_PERIMETER_RES = 60
        self.FARSITE_MIN_IGNITION_VERTEX_DISTANCE = 15.0
        self.FARSITE_SPOT_GRID_RESOLUTION = 60.0
        self.FARSITE_SPOT_PROBABILITY = 0  # 0.9
        self.FARSITE_SPOT_IGNITION_DELAY = 0
        self.FARSITE_MINIMUM_SPOT_DISTANCE = 60
        self.FARSITE_ACCELERATION_ON = 1
        self.FARSITE_FILL_BARRIERS = 1
        self.SPOTTING_SEED = 253114
        
        self.FUEL_MOISTURES_DATA = [[0, 3, 4, 6, 30, 60]]
        
        self.RAWS_ELEVATION = 2501
        self.RAWS_UNITS = 'English'
        # Add self.raws from the init
              
        self.FOLIAR_MOISTURE_CONTENT = 100
        self.CROWN_FIRE_METHOD = 'ScottReinhardt'
        
        self.WRITE_OUTPUTS_EACH_TIMESTEP = 0
        
        self.temperature = 66
        self.humidity = 8
        self.precipitation = 0
        self.cloudcover = 0
        
    def tostring(self):
        config_text = 'FARSITE INPUTS FILE VERSION {}\n'.format(self.version)
        
        str_start = '{month} {day} {time}'.format(
                            month = self.FARSITE_START_TIME.month,
                            day = self.FARSITE_START_TIME.day,
                            time = '{}{:02d}'.format(
                                    self.FARSITE_START_TIME.hour,
                                    self.FARSITE_START_TIME.minute))
        config_text += 'FARSITE_START_TIME: {}\n'.format(str_start)

        str_end = '{month} {day} {time}'.format(
                            month = self.FARSITE_END_TIME.month,
                            day = self.FARSITE_END_TIME.day,
                            time = '{}{:02d}'.format(
                                    self.FARSITE_END_TIME.hour,
                                    self.FARSITE_END_TIME.minute))
        config_text += 'FARSITE_END_TIME: {}\n'.format(str_end)
        
        config_text += 'FARSITE_TIMESTEP: {}\n'.format(self.FARSITE_TIMESTEP)
        config_text += 'FARSITE_DISTANCE_RES: {}\n'.format(self.FARSITE_DISTANCE_RES)
        config_text += 'FARSITE_PERIMETER_RES: {}\n'.format(self.FARSITE_PERIMETER_RES)
        config_text += 'FARSITE_MIN_IGNITION_VERTEX_DISTANCE: {}\n'.format(self.FARSITE_MIN_IGNITION_VERTEX_DISTANCE)
        config_text += 'FARSITE_SPOT_GRID_RESOLUTION: {}\n'.format(self.FARSITE_SPOT_GRID_RESOLUTION)
        config_text += 'FARSITE_SPOT_PROBABILITY: {}\n'.format(self.FARSITE_SPOT_PROBABILITY)
        config_text += 'FARSITE_SPOT_IGNITION_DELAY: {}\n'.format(self.FARSITE_SPOT_IGNITION_DELAY)
        config_text += 'FARSITE_MINIMUM_SPOT_DISTANCE: {}\n'.format(self.FARSITE_MINIMUM_SPOT_DISTANCE)
        config_text += 'FARSITE_ACCELERATION_ON: {}\n'.format(self.FARSITE_ACCELERATION_ON)
        config_text += 'FARSITE_FILL_BARRIERS: {}\n'.format(self.FARSITE_FILL_BARRIERS)
        config_text += 'SPOTTING_SEED: {}\n'.format(self.SPOTTING_SEED)
        
        # Fuel moistures
        config_text += 'FUEL_MOISTURES_DATA: {}\n'.format(len(self.FUEL_MOISTURES_DATA))
        for data in self.FUEL_MOISTURES_DATA:
            config_text += '{} {} {} {} {} {}\n'.format(data[0], data[1], data[2],
                                                      data[3], data[4], data[5])
            
        config_text += 'RAWS_ELEVATION: {}\n'.format(self.RAWS_ELEVATION)
        config_text += 'RAWS_UNITS: {}\n'.format(self.RAWS_UNITS)
        
        # Weather data (currently only a single weather data)
        config_text += 'RAWS: 1\n'
        config_text += '{year} {month} {day} {time} {temperature} {humidity} {precipitation} {windspeed} {winddirection} {cloudcover}\n'.format(
                                year = self.FARSITE_START_TIME.year,
                                month = self.FARSITE_START_TIME.month,
                                day = self.FARSITE_START_TIME.day,
                                time = '{}{:02d}'.format(
                                    self.FARSITE_START_TIME.hour, 
                                    self.FARSITE_START_TIME.minute),
                                temperature = self.temperature,
                                humidity = self.humidity,
                                precipitation = self.precipitation,
                                windspeed = self.windspeed,
                                winddirection = self.winddirection,
                                cloudcover = self.cloudcover
                            )
        config_text += 'FOLIAR_MOISTURE_CONTENT: {}\n'.format(self.FOLIAR_MOISTURE_CONTENT)
        config_text += 'CROWN_FIRE_METHOD: {}\n'.format(self.CROWN_FIRE_METHOD)
        config_text += 'WRITE_OUTPUTS_EACH_TIMESTEP: {}'.format(self.WRITE_OUTPUTS_EACH_TIMESTEP)
        
        return config_text


class Run_File:
    def __init__(self, lcppath: str, cfgpath: str, ignitepath: str, barrierpath: str, outpath: str):
        self.lcppath = lcppath
        self.cfgpath = cfgpath
        self.ignitepath = ignitepath
        self.barrierpath = barrierpath
        self.outpath = outpath

    def tostring(self):
        return '{lcpath} {cfgpath} {ignitepath} {barrierpath} {outpath} -1'.format(
                                lcpath =  self.lcppath, 
                                cfgpath = self.configpath, 
                                ignitepath = self.ignitepath, 
                                barrierpath = self.barrierpath, 
                                outpath = self.outpath)


def generate_landscape(geom_5070: Polygon, description='test'):
    bounds = geom_5070.bounds

    ulx = bounds[0]-10000
    uly = bounds[3]+10000
    lrx = bounds[2]+10000
    lry=  bounds[1]-10000

    fname_lst = {'density': 'US_140CBD_12052016/Grid/us_140cbd', 
                 'base': 'US_140CBH_12052016/Grid/us_140cbh', 
                 'cover': 'US_140CC_12052016/Grid/us_140cc', 
                 'height': 'US_140CH_12052016/Grid/us_140ch', 
                 'fuel': 'US_140FBFM40_20180618/Grid/us_140fbfm40', 
                 'aspect': 'Aspect/Grid/us_asp', 
                 'elevation': 'DEM_Elevation/Grid/us_dem', 
                 'slope': 'Slope/Grid/us_slp'}
    type_lst = {'density': 'cbd',
                'base': 'cbh',
                'cover': 'cc',
                'height': 'ch',
                'fuel': 'fuel',   # fbfm40
                'aspect': 'aspect',
                'elevation': 'elevation', #dem
                'slope': 'slope'}

    from_folder = os.path.join('/data', 'firemap', 'landfire', 'mosaic')
    to_folder = os.path.join(os.getenv('HOME'), 'farsite-devAPI', 'inputs', 'landscapes')

    # Create the asc files
    ascpath_lst = {}
    for (key, fname) in fname_lst.items():
        ascpath_lst[key] = f'{os.path.join(to_folder, description)}-{key}.asc'
        os.system(f'gdal_translate -of AAIGrid -a_nodata -32768 -projwin {ulx} {uly} {lrx} {lry} {os.path.join(from_folder, fname)} {ascpath_lst[key]}')
        
    lcppath = os.path.join(to_folder, description)
    lcpmakepath = os.path.join(os.getenv('HOME'), 'farsite-devAPI', 'src', 'lcpmake')

    base_command = f'{lcpmakepath} -latitude {geom_5070.centroid.y} -landscape {lcppath}'
    run_command = base_command
    for (key, ascpath) in ascpath_lst.items():
        run_command += f' -{key} {ascpath}'

    os.system(run_command)
    
    os.system(f"rm {os.path.join(to_folder, '*asc')} {os.path.join(to_folder, '*xml')}")
    
    return lcppath



class Farsite:
    def __init__(self, initial: Polygon, parameters: dict, lcppath: str = None):
        # Setup files
        self.lcppath = lcppath

        if lcppath == None:
            raise ValueErorr(f'The filepath {lcppath} cannot be used for lcppath')

        start_dt = datetime.datetime(year=2019, month=1, day=1, hour=10, minute=0)
        end_dt = start_dt + params['dt']
        windspeed = params['windspeed']
        winddirection = params['winddirection']
        
        self.config = Config_File(startdt, enddt, windspeed, winddirection)

        
        # Generate RunFile

    def run(self):
        raise NotImplementedError('Farsite:run')
        # Run farsite
        # Returns final polygon

In [31]:
poly, dt = get_observation('Maria2019', 0)
lcppath = generate_landscape(poly)

ERROR 4: /data/firemap/landfire/mosaic/US_140CBD_12052016/Grid/us_140cbd: No such file or directory
ERROR 4: /data/firemap/landfire/mosaic/US_140CBH_12052016/Grid/us_140cbh: No such file or directory
ERROR 4: /data/firemap/landfire/mosaic/US_140CC_12052016/Grid/us_140cc: No such file or directory
ERROR 4: /data/firemap/landfire/mosaic/US_140CH_12052016/Grid/us_140ch: No such file or directory




FILE READ ERROR:

File Not Found /home/jovyan/farsite-devAPI/inputs/landscapes/test-elevation.asc


ERROR 4: /data/firemap/landfire/mosaic/US_140FBFM40_20180618/Grid/us_140fbfm40: No such file or directory
ERROR 4: /data/firemap/landfire/mosaic/Aspect/Grid/us_asp: No such file or directory
ERROR 4: /data/firemap/landfire/mosaic/DEM_Elevation/Grid/us_dem: No such file or directory
ERROR 4: /data/firemap/landfire/mosaic/Slope/Grid/us_slp: No such file or directory
rm: cannot remove '/home/jovyan/farsite-devAPI/inputs/landscapes/*asc': No such file or directory
rm: cannot remove '/home/jovyan/farsite-devAPI/inputs/landscapes/*prj': No such file or directory
rm: cannot remove '/home/jovyan/farsite-devAPI/inputs/landscapes/*xml': No such file or directory


1. Start a data assimilation workflow
    a. Generate a landscape file, assign a descriptor
    b. 
2. testing

In [16]:
start_dt = datetime.datetime(year=2023, month=5, day=2, hour=15, minute=0)
end_dt = start_dt + datetime.timedelta(minutes=32)
ws = 10
wd = 0

config = Config_File(start_dt, end_dt, ws, wd)

In [17]:
print(config.tostring())

FARSITE INPUTS FILE VERSION 1.0
FARSITE_START_TIME: 9 9 1900
FARSITE_END_TIME: 9 9 1932
FARSITE_TIMESTEP: 32
FARSITE_DISTANCE_RES: 30
FARSITE_PERIMETER_RES: 60
FARSITE_MIN_IGNITION_VERTEX_DISTANCE: 15.0
FARSITE_SPOT_GRID_RESOLUTION: 60.0
FARSITE_SPOT_PROBABILITY: 0
FARSITE_SPOT_IGNITION_DELAY: 0
FARSITE_MINIMUM_SPOT_DISTANCE: 60
FARSITE_ACCELERATION_ON: 1
FARSITE_FILL_BARRIERS: 1
SPOTTING_SEED: 253114
FUEL_MOISTURES_DATA: 1
0 3 4 6 30 60
RAWS_ELEVATION: 2501
RAWS_UNITS: English
RAWS: 1
2019 9 9 1900 66 8 0 10 0 0
FOLIAR_MOISTURE_CONTENT: 100
CROWN_FIRE_METHOD: ScottReinhardt
WRITE_OUTPUTS_EACH_TIMESTEP: 0
